# Paso 2: Generación de vectores TF-IDF en PySpark
Notebook completo con carga de libros, preparación y cálculo de TF‑IDF.

In [2]:
import os
os.environ["PYSPARK_SUBMIT_ARGS"] = "--driver-memory 6g --executor-memory 6g pyspark-shell"

### Tokenizer separa texto y los combierte palabras, StopWordsRemover quita palabras comunes(the, and, a, etc.), HashingTF covierte los tokens en numeros usando terme frecuency

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name, regexp_replace, lower
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF

spark = SparkSession.builder.appName('TFIDF_Libros').getOrCreate()

## Cargar libros desde carpeta `books/`

In [14]:
# Assuming your files are .txt files inside the 'books' directory:
df = spark.read.text("//wsl.localhost/ubuntu/home/jayeoli/100Books/books/*.txt").withColumn('filename', input_file_name())
df.show(5)

25/12/08 21:03:11 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: //wsl.localhost/ubuntu/home/jayeoli/100Books/books/*.txt.
org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "null"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3581)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3612)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:172)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3716)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3667)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:557)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:366)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:55)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:381)
	at org.apache.spark.sql.catalyst.analysis.ResolveData

Py4JJavaError: An error occurred while calling o108.text.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "null"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3581)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3612)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:172)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3716)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3667)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:557)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:366)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:777)
	at scala.collection.immutable.List.map(List.scala:247)
	at scala.collection.immutable.List.map(List.scala:79)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(DataSource.scala:775)
	at org.apache.spark.sql.execut

## Unir el contenido por archivo (un documento por libro). 'collect_list' junta valors de varias filas adentros una lista. 'concat_ws' es el ayuda con los espacios

In [ ]:
from pyspark.sql.functions import collect_list, concat_ws
df_docs = df.groupBy('filename').agg(concat_ws(' ', collect_list('value')).alias('text'))
df_docs.show(3)

[Stage 8:>                                                          (0 + 4) / 4]

## Limpieza básica de texto

### Quita caracteres que no se necesita, todo el texto se convierte en minusculas. Crea una nueva columna que se llama 'clean'

In [6]:
df_clean = df_docs.withColumn('clean', lower(regexp_replace('text', '[^a-zA-Záéíóúñ ]', ' ')))
df_clean.show(3)

+--------------------+--------------------+--------------------+
|            filename|                text|               clean|
+--------------------+--------------------+--------------------+
|file:///home/robc...|*** START OF THE ...|    start of the ...|
|file:///home/robc...|*** START OF THE ...|    start of the ...|
|file:///home/robc...|The Project Guten...|the project guten...|
+--------------------+--------------------+--------------------+
only showing top 3 rows


## Tokenización

### Toma la columna 'clean' y aguarda el nuevo en la nueva columna

In [7]:
tokenizer = Tokenizer(inputCol='clean', outputCol='tokens')
df_tok = tokenizer.transform(df_clean)
df_tok.select('tokens').show(3, truncate=50)

+--------------------------------------------------+
|                                            tokens|
+--------------------------------------------------+
|[, , , , start, of, the, project, gutenberg, eb...|
|[, , , , start, of, the, project, gutenberg, eb...|
|[the, project, gutenberg, ebook, of, a, tale, o...|
+--------------------------------------------------+
only showing top 3 rows


## Remover Stopwords

### Usa la lista tokens para filtrar los stopwords

In [8]:
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')
df_filt = remover.transform(df_tok)
df_filt.select('filtered').show(3, truncate=50)

[Stage 13:>                                                         (0 + 1) / 1]

+--------------------------------------------------+
|                                          filtered|
+--------------------------------------------------+
|[, , , , start, project, gutenberg, ebook, , , ...|
|[, , , , start, project, gutenberg, ebook, , , ...|
|[project, gutenberg, ebook, tale, two, cities, ...|
+--------------------------------------------------+
only showing top 3 rows


## Calcular TF 

### HashingTF,mapea cada palabra en unos 20000 poscisiones de vectores

In [9]:
tf = HashingTF(inputCol='filtered', outputCol='tf', numFeatures=20000)
df_tf = tf.transform(df_filt)
df_tf.select('tf').show(3)

[Stage 16:>                                                         (0 + 1) / 1]

+--------------------+
|                  tf|
+--------------------+
|(20000,[0,1,2,3,6...|
|(20000,[0,1,2,6,9...|
|(20000,[1,2,4,5,6...|
+--------------------+
only showing top 3 rows


## Calcular IDF y obtener TF‑IDF final

### Usa est equacion, TF-IDF = Term Frequency × Inverse Document Frequency. aqui es donde saca la importancia de cada palabra en el documento

In [22]:
idf = IDF(inputCol='tf', outputCol='tfidf')
idf_model = idf.fit(df_tf)
df_tfidf = idf_model.transform(df_tf)
df_tfidf.select('filename', 'tfidf').show(10)

[Stage 45:>                                                         (0 + 1) / 1]

+--------------------+--------------------+
|            filename|               tfidf|
+--------------------+--------------------+
|file:///home/robc...|(20000,[0,1,2,3,6...|
|file:///home/robc...|(20000,[0,1,2,6,9...|
|file:///home/robc...|(20000,[1,2,4,5,6...|
|file:///home/robc...|(20000,[7,8,9,13,...|
|file:///home/robc...|(20000,[1,11,13,1...|
|file:///home/robc...|(20000,[0,2,4,6,7...|
|file:///home/robc...|(20000,[2,3,4,11,...|
|file:///home/robc...|(20000,[2,6,15,16...|
|file:///home/robc...|(20000,[0,9,11,12...|
|file:///home/robc...|(20000,[1,2,3,11,...|
+--------------------+--------------------+
only showing top 10 rows


### extrae el nombre de la carpeta y remplasa el %20 con un espacio,y enseña los nombres de los libros que se limpiaron

In [28]:
from pyspark.sql.functions import regexp_replace, regexp_extract, col

df_tfidf.select(
    regexp_replace(
        regexp_extract(col("filename"), r"([^/]+)$", 1),
        "%20", " "
    ).alias("libro")
).show(truncate=False)

+---------------------------------------------+
|libro                                        |
+---------------------------------------------+
|Bleak House.txt                              |
|The Count of Monte Cristo.txt                |
|History of Tom Jones, a Foundling.txt        |
|The Complete Works of William Shakespeare.txt|
|War and Peace.txt                            |
|The Brothers Karamazov.txt                   |
|Anna Karenina.txt                            |
|Don Quixote.txt                              |
|A Military Dictionary and Gazetteer.txt      |
|The Republic.txt                             |
|My Life — Volume 1.txt                       |
|The Iliad.txt                                |
|Little Women; Or, Meg, Jo, Beth, and Amy.txt |
|Twenty years after.txt                       |
|Crime and Punishment.txt                     |
|Leviathan.txt                                |
|Middlemarch.txt                              |
|Moby Dick; Or, The Whale.txt           

In [29]:
df_tfidf.select("filename", "tfidf").write.mode("overwrite").parquet("tfidf_vectors.parquet")